In [11]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


In [12]:
from sklearn.model_selection import train_test_split

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    
    word_count = len(sentence.split(" "))  # 15단어가 넘는 문장을 공백으로 변환
    if word_count > 14 :
        return ""
    
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

corpus = []
for sentence in raw_corpus:    
    if len(sentence) == 0: continue # 
       
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence) == 0: continue # preprocess_sentence() 에서 공백으로 만들어준 단어들을 목록에 넣지 않고 continue 한다
    corpus.append(preprocessed_sentence)

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:] 
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.20, random_state=33)

[[  2  62 274 ...   0   0   0]
 [  2 115   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  46 ...   0   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 642 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd0e1f32e10>


In [13]:
print("Source Train:", enc_train.shape)
print(enc_train[:3]) 
print("Target Train:", dec_train.shape)

Source Train: (128379, 15)
[[   2   45   80   68   11   55   47    3    0    0    0    0    0    0
     0]
 [   2    8   13  995   37   31  296 4001   18   11    3    0    0    0
     0]
 [   2    9 2001   19 2263   17  500 1282   42  648    3    0    0    0
     0]]
Target Train: (128379, 15)


데이터의 열 개수가 지나치게 크게 나오는 문제가 있었습니다, 이를 해결하기 위해 print(enc_train[:3])로 데이터의 예시를 확인해보았습니다.

In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
626/626 [==============================] - 214s 332ms/step - loss: 3.7541
Epoch 2/10
626/626 [==============================] - 209s 334ms/step - loss: 2.8744
Epoch 3/10
626/626 [==============================] - 210s 336ms/step - loss: 2.6835
Epoch 4/10
626/626 [==============================] - 211s 337ms/step - loss: 2.5404
Epoch 5/10
626/626 [==============================] - 207s 330ms/step - loss: 2.4324
Epoch 6/10
626/626 [==============================] - 203s 323ms/step - loss: 2.3342
Epoch 7/10
626/626 [==============================] - 202s 322ms/step - loss: 2.2511
Epoch 8/10
626/626 [==============================] - 201s 321ms/step - loss: 2.1649
Epoch 9/10
626/626 [==============================] - 202s 321ms/step - loss: 2.0856
Epoch 10/10
626/626 [==============================] - 202s 322ms/step - loss: 2.0066


학습과정에서 지나치게 시간이 오래 걸리는 현상이 발생하였고 이를 해결하기 위해 GPU의 작동을 확인하고 데이터의 의미없는 부분을 줄여보았습니다.
최종적으로 토큰이 15개 이상인 문장들을 리스트에서 제거하고 모델의 epochs를 30 -> 10으로 줄인 결과 적절한 속도로 모델을 학습시킬 수 있었습니다. 

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

In [18]:
generate_text(lyricist, tokenizer, init_sentence="<start> if i ", max_len=20)

'<start> if i could turn back , <end> '

In [26]:
generate_text(lyricist, tokenizer, init_sentence="<start> love you", max_len=20)

'<start> love you , love you <end> '

In [27]:
generate_text(lyricist, tokenizer, init_sentence="<start> i believe", max_len=20)

'<start> i believe in god , i m not gonna be <end> '

In [28]:
generate_text(lyricist, tokenizer, init_sentence="<start> when i", max_len=20)

'<start> when i m in the club <end> '

In [32]:
generate_text(lyricist, tokenizer, init_sentence="<start> dont worry", max_len=20)

'<start> dont worry about the way i feel like i m a flirt <end> '

In [35]:
generate_text(lyricist, tokenizer, init_sentence="<start> know that", max_len=20)

'<start> know that i m a motherfucking monster <end> '

In [49]:
generate_text(lyricist, tokenizer, init_sentence="<start> and in our time", max_len=30)

'<start> and in our time of love <end> '

텍스트를 생성한 결과 제법 그럴싸한 가사가 생성되엇습니다.
시작 문장을 다른 문장으로 바꿔보았을때에도 꽤 괜찮은 가사가 생성되는 것을 확인하였습니다.